<a href="https://colab.research.google.com/github/rofikoab1603/DS-13/blob/main/Data_Science_2_DS13_(Rofiko).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math

!pip install gspread==5.3.2
import gspread
from google.oauth2.service_account import Credentials

import warnings
warnings.filterwarnings('ignore')

  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    '/content/ds12-rofiko-1ed718f09c58.json',
    scopes=scopes)

gc = gspread.authorize(credentials)


In [ ]:
file = gc.open('Raw Parkiran PT. SUMO UTOMO')

name_sheet = ['Jan-2021', 'Feb-2021', 'Mar-2021', 'Apr-2021', 'Mei-2021', 'Jun-2021',
              'Jul-2021', 'Agu-2021', 'Sep-2021', 'Okt-2021', 'Nov-2021', 'Des-2021']

data = pd.DataFrame(columns=['Gate','Vehicle','Entry','Exit'])
for i in name_sheet:
  sheet = file.worksheet(i)
  df = pd.DataFrame(sheet.get_all_records())
  data = data.append(df) 

data['Entry'] = pd.to_datetime(data['Entry'])
data['Exit'] = pd.to_datetime(data['Exit'])
data['Month'] = data['Entry'].dt.month_name()


In [ ]:
data.head(10)

,Gate,Vehicle,Entry,Exit,Month
0,B2,Motorcycle,2021-01-06 22:23:57,2021-01-19 23:45:02,January
1,B3,Car,2021-01-07 05:43:32,2021-01-06 04:36:17,January
2,A1,Car,2021-01-06 04:47:47,2021-01-24 00:58:50,January
3,B1,Motorcycle,2021-01-05 09:44:31,2021-01-04 20:12:20,January
4,B2,Car,2021-01-21 04:20:00,2021-01-16 13:29:57,January
5,B1,Car,2021-01-26 12:44:59,2021-01-20 18:07:24,January
6,B1,Car,2021-01-18 05:29:22,2021-01-20 16:42:29,January
7,B1,Motorcycle,2021-01-21 13:39:34,2021-01-01 08:23:46,January
8,B1,Car,2021-01-17 15:11:33,2021-01-04 22:23:22,January
9,A1,Car,2021-01-18 05:37:43,2021-01-21 10:05:17,January


## Persentase transaksi error tiap bulan dan keseluruhan

In [ ]:
data['Validasi'] = data['Entry'] < data['Exit']
bulan = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
laporan_1 = data.groupby(['Month','Validasi']).agg({'Entry':'count'}).unstack().reset_index()
laporan_1.columns = ['Month', 'False', 'True']
laporan_1['Persentase'] = (laporan_1['False']/(laporan_1['False']+laporan_1['True']))*100
laporan_1.index = pd.CategoricalIndex(laporan_1['Month'], categories=bulan, ordered=True)
laporan_1 = laporan_1.sort_index().reset_index(drop=True)
laporan_1.to_excel('Laporan_1.xlsx', index=False)
laporan_1

,Month,False,True,Persentase
0,January,48,52,48.0
1,February,46,54,46.0
2,March,46,54,46.0
3,April,53,47,53.0
4,May,41,59,41.0
5,June,54,46,54.0
6,July,55,45,55.0
7,August,54,46,54.0
8,September,54,46,54.0
9,October,50,50,50.0


In [ ]:
data.info()

## Pendapatan transaksi tiap bulan

In [ ]:
data_1 = data[data['Validasi'] == True].reset_index()

def get_hours(end,start):
  td = (end-start)
  minutes = td.total_seconds()/60
  hours = minutes/60
  days = hours/24

  return math.ceil(hours)

for index, row in data_1.iterrows():
  data_1.loc[index,'Duration'] = get_hours(row['Exit'], row['Entry'])

data_1.loc[data_1['Vehicle'] == 'Car', 'Price'] = data_1['Duration']*3000
data_1.loc[data_1['Vehicle'] == 'Motorcycle', 'Price'] = data_1['Duration']*2000
laporan_2 = data_1.groupby(['Month']).agg({'Price':'sum'}).reset_index()
laporan_2.index = pd.CategoricalIndex(laporan_2['Month'], categories=bulan, ordered=True)
laporan_2 = laporan_2.sort_index().reset_index(drop=True)
laporan_2.to_excel('Laporan_2.xlsx', index=False)
laporan_2

,Month,Price
0,January,29871000.0
1,February,29484000.0
2,March,31814000.0
3,April,30853000.0
4,May,41624000.0
5,June,23418000.0
6,July,19737000.0
7,August,30605000.0
8,September,24206000.0
9,October,32611000.0


## Pendapatan tiap bulan berdasarkan jenis kendaraan

In [ ]:
kendaraan = ['Car', 'Motorcyle']
laporan_3 = data_1.groupby(['Month', 'Vehicle']).agg({'Price':'sum'}).reset_index()
laporan_3.index = pd.CategoricalIndex(laporan_3['Month'], categories=bulan, ordered=True)
laporan_3 = laporan_3.sort_index().reset_index(drop=True)
laporan_3.to_excel('Laporan_3.xlsx', index=False)
laporan_3

,Month,Vehicle,Price
0,January,Car,13065000.0
1,January,Motorcycle,16806000.0
2,February,Car,13974000.0
3,February,Motorcycle,15510000.0
4,March,Motorcycle,12620000.0
5,March,Car,19194000.0
6,April,Car,18351000.0
7,April,Motorcycle,12502000.0
8,May,Motorcycle,13058000.0
9,May,Car,28566000.0


In [ ]:
pd.pivot_table(laporan_3, index=['Vehicle'], columns=['Month']).reset_index()

Vehicle       Price                                                  \
Month                   April      August    December    February     January   
0             Car  18351000.0  14367000.0  23067000.0  13974000.0  13065000.0   
1      Motorcycle  12502000.0  16238000.0   7352000.0  15510000.0  16806000.0   

                                                                               \
Month        July        June       March         May    November     October   
0      11037000.0  16062000.0  19194000.0  28566000.0  20025000.0  19767000.0   
1       8700000.0   7356000.0  12620000.0  13058000.0  12294000.0  12844000.0   

                   
Month   September  
0      13734000.0  
1      10472000.0